# Virtual Source - Efficiency of Boost & Buck - Converters

eta = P_OUT / P_IN = (V_OUT * I_OUT) / (V_IN * I_IN)

## Boost Converter

The general circuit can be seen on [wikipedia](https://en.wikipedia.org/wiki/Boost_converter#Circuit_analysis).

There are two states for the continuous mode:

- ON - where the mosfet-switch is closed, so the input is shortcut with the coil
- OFF - where the mosfet-switch is open and the energy of the coil is pumped to the output-capacitor

To model the efficiency there must be added some more resistive components

### Resistive components

Efficiency is mostly through heat by resistive parts in the circuit. Components to explore:

- coil
- mosfet
- diode
- capacitor
- traces

Starting with the [datasheet of the BQ25570](https://www.ti.com/product/BQ25570) and the accompanying [user's guide for the evaluation module](https://www.ti.com/tool/BQ25570EVM-206) for the investigation.

#### Coil 

The Coil for the EVM is `LPS4018-223M` with 22uH.

In [ ]:
L_boost_coil = 22e-6
R_boost_coil = 0.360
I_boost_coil_rms = 0.65
I_boost_coil_sat = 0.74

#### Diode & Mosfet

The diode of the boost-circuit is replaced by mosfets for the BQ-ICs.
Looking at the datasheet there are

- a low side charge pnp-mosfet (the general mosfet of a boost circuit)
- a high side charge npn-mosfet (replacement for the diode)
- another npn-mosfet for the battery (pin, where the storage capacity is)

Conduction loss can be calculated by:

```
P_C = (I_D)² * R_DS
```

Several on-resistances can be found:

In [ ]:
# VBat = 4.2 V
R_ds_chg_ls_typ = 0.7  # low side, typical
R_ds_chg_ls_max = 0.9
R_ds_chg_hs_typ = 2.3  # high side
R_ds_chg_hs_max = 3.0
# VBat = 2.1 V
R_ds_chg_ls_typ = 0.8  # low side, typical
R_ds_chg_ls_max = 1.0
R_ds_chg_hs_typ = 3.7  # high side
R_ds_chg_hs_max = 4.8

# as the main cap is
R_DS_Bat_typ = 0.95  # typical
R_DS_Bat_max = 1.5

#### Capacitor

For the Capacitor some parameters can be determined. 
The EVM uses a `GRM43SR60J107ME20L`, 100uF, 6.3volts, X5R, 20%

### Switching losses

This is mostly guesswork, as the datasheet does not give much insight. 

**NOTE**: The gate-charge of the mosfets is only assumed by looking at similar discrete mosfets (similar current rating for that r_ds-range).

```
P_SW = V_IN * I_OUT * f_SW * (Q_GS + Q_GD) / I_G
```

or alternatively

```math
P_SW = 1/2 * V_IN * I_D * (t_R + t_F) * f_SW
```

Unfortunately the rise/fall-times or gate-current can't be determined. 

The additional Gate-Charge loss is

```
P_GC = Q_G * V_GS * f_SW
```

In [1]:
f_sw_boost_max = 1e6
Q_gate_pnp = 1.0e-9  # estimated, .36 ... 1.4 nC for similar fets
Q_gate_npn = 1.5e-9  # estimated, 1.2 ... 1.9 nC for similar fets

# example losses
P_GC = Q_gate_pnp * 4.2 * f_sw_boost_max

### IC Consumption

The datasheet gives info about the current-draw of the IC. Referenced Voltage should be V_storage.


In [ ]:
I_bq70_quiescent = 488e-9  # max 700 nA @ ~ 25°C